In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [6]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.


In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.


In [7]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, youtube.com, alura.com.br



In [8]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [22]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [23]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [24]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [25]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Atue como um Líder experiente em Security Awareness. Sua tarefa é realizar uma busca
        abrangente no Google Search pelas últimas notícias sobre golpes, fraudes e ataques de
        cibersegurança e segurança da informação.

        Você deve:
      - Verificar um máximo de [Número] resultados de busca. (Sugestão inicial: 10)
      - Concentrar-se em notícias publicadas nos últimos 45 dias (ou seja, a partir de aproximadamente 2 de abril de 2025 até hoje, 17 de maio de 2025).
      - Priorizar fontes confiáveis e relevantes na área de segurança da informação.
      - Fornecer um resumo conciso de cada notícia encontrada, incluindo o título, a fonte e a data de publicação.
      - Listar os links diretos para cada uma das notícias encontradas.

      O objetivo é fornecer ao próximo agente uma lista de notícias relevantes e atuais para análise.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [18]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em análise de conteúdo de Security Awareness. Dada a seguinte lista de notícias (fornecida pelo Buscador de Notícias),
        sua tarefa é selecionar a notícia mais relevante e impactante para a criação de um post de conscientização para os colaboradores de uma empresa.

        Considere os seguintes critérios para a seleção:
        - **Atualidade:** A notícia deve ser recente e refletir ameaças ativas no cenário atual.
        - **Impacto Potencial:** A notícia deve abordar um golpe, fraude ou ataque que possa afetar um número significativo de pessoas ou causar danos relevantes.
        - **Clareza e Compreensão:** A notícia deve ser clara e fácil de entender, mesmo para pessoas sem conhecimento técnico em segurança da informação.
        - **Relevância para o público:** A notícia deve ser algo que os colaboradores da empresa provavelmente encontrarão ou precisarão estar cientes.

        Sua saída deve ser:
        - O título da notícia selecionada.
        - O link direto para a notícia selecionada.
        - Um breve parágrafo explicando o motivo da sua seleção, destacando sua relevância e potencial impacto para os colaboradores.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [19]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um redator experiente em comunicação de Security Awareness, com foco em criar mensagens claras, objetivas e
            impactantes para colaboradores de uma empresa. Com base na seguinte notícia selecionada (fornecida pelo Selecionador de Notícias),
            sua tarefa é criar um post conciso e informativo, seguindo os seguintes tópicos:

          - **Título:** Crie um título chamativo e direto, que capture a atenção do leitor e resuma o tema principal da notícia.
          - **Introdução Sucinta:** Escreva um breve parágrafo (máximo 3-4 frases) introduzindo o golpe, fraude ou ataque de forma
          clara e direta, explicando o que aconteceu.
          - **Riscos:** Detalhe os principais riscos e consequências que os colaboradores podem enfrentar caso caiam nesse tipo de golpe ou ataque.
          Seja específico sobre os impactos potenciais (perda de dados, prejuízo financeiro, acesso indevido a contas, etc.).
          - **Como se Proteger:** Forneça dicas práticas e acionáveis sobre como os colaboradores podem se proteger contra esse tipo de ameaça.
          Use linguagem clara e direta, com passos simples e fáceis de seguir.

          Seu objetivo é criar um post que informe, alerte e oriente os colaboradores sobre como se manterem seguros.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [20]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um revisor de conteúdo de Security Awareness com um olhar crítico e foco na clareza e relevância da informação.
            Sua tarefa é revisar o post gerado pelo Redator de Comunicação, com base na notícia original (fornecida pelo Selecionador de Notícias).

            Você deve avaliar se o post:
            - **Faz sentido:** As informações apresentadas são coerentes e alinhadas com a notícia original?
            - **É preciso:** Os detalhes do golpe, fraude ou ataque estão corretos?
            - **É claro e objetivo:** A linguagem utilizada é fácil de entender por todos os colaboradores?
            - **O título é adequado:** O título captura a essência da notícia e é chamativo?
            - **A introdução é eficaz:** A introdução contextualiza o assunto de forma clara e concisa?
            - **Os riscos estão bem definidos:** Os potenciais impactos para os colaboradores são claros e relevantes?
            - **As dicas de proteção são práticas e acionáveis:** Os colaboradores saberão o que fazer para se proteger?
            - **O tom é adequado:** O tom da mensagem é informativo e de alerta, sem gerar pânico desnecessário?

            Se necessário, sugira melhorias ou correções para o post, visando garantir a máxima clareza e impacto da comunicação.
            Sua saída deve ser um feedback detalhado sobre a qualidade do post e, se aplicável, sugestões de melhoria.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")
